In [2]:
!pip install pandas --user

  Using cached https://files.pythonhosted.org/packages/08/01/803834bc8a4e708aedebb133095a88a4dad9f45bbaf5ad777d2bea543c7e/pandas-0.22.0.tar.gz
  Installing build dependencies ... error
  Complete output from command /usr/bin/python3.4 -m pip install --ignore-installed --no-user --prefix /tmp/pip-build-env-_bjf3bnd https://files.pythonhosted.org/packages/81/30/e935244ca6165187ae8be876b6316ae201b71485538ffac1d718843025a9/wheel-0.31.1-py2.py3-none-any.whl#sha256=80044e51ec5bbf6c894ba0bc48d26a8c20a9ba629f4ca19ea26ecfcf87685f5f https://files.pythonhosted.org/packages/ff/f4/385715ccc461885f3cedf57a41ae3c12b5fec3f35cce4c8706b1a112a133/setuptools-40.0.0-py2.py3-none-any.whl#sha256=d68abee4eed409fbe8c302ac4d8429a1ffef912cd047a903b5701c024048dd49 https://files.pythonhosted.org/packages/b6/dc/f67cd160d763f052113a098938af9adc528f8dc0203d2fc89e2e4e897d3a/Cython-0.28.4-cp34-cp34m-manylinux1_x86_64.whl#sha256=f16a87197939977824609005b73f9ebb291b9653a14e5f27afc1c5d6f981ba39 https://files.pythonhosted.

In [3]:
import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # for 3d plotting

import h5py

%matplotlib inline

Using TensorFlow backend.


In [5]:


# load the data
with h5py.File('../data/full_dataset_vectors.h5', 'r') as hf:
    x_train_raw = hf["X_train"][:]
    y_train_raw = hf["y_train"][:]
    x_test_raw = hf["X_test"][:]
    y_test_raw = hf["y_test"][:]

In [6]:
# length check
assert(len(x_train_raw) == len(y_train_raw))
assert(len(x_test_raw) == len(y_test_raw))

In [7]:
# 1D vector to rgb values, provided by ../input/plot3d.py
def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:,:-1]

# Transform data from 1d to 3d rgb
def rgb_data_transform(data):
    data_t = []
    for i in range(data.shape[0]):
        data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
    return np.asarray(data_t, dtype=np.float32)

In [8]:
n_classes = 10 # from 0 to 9, 10 labels totally

x_train = rgb_data_transform(x_train_raw)
x_test = rgb_data_transform(x_test_raw)

y_train = to_categorical(y_train_raw, n_classes)
y_test = to_categorical(y_test_raw, n_classes)

In [16]:
from tensorflow.keras.layers import BatchNormalization, Dense,Dropout, Conv3D, MaxPooling3D, Flatten

In [17]:
from tensorflow.keras.optimizers import SGD, Adam

In [12]:
model = tf.keras.Sequential()


In [19]:
model.add(Conv3D(16, (3,3,3) ,activation='relu'))
model.add(Conv3D(32, (3,3,3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2),strides=2))

model.add(Conv3D(64, (3,3,3) ,activation='relu'))
model.add(Conv3D(128, (3,3,3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=2))

model.add(BatchNormalization(trainable=True))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(10, activation='softmax'))

adam = Adam(lr=0.05)

model.compile(loss='categorical_crossentropy', optimizer=adam)

model.fit(x_train, y_train, batch_size=2, epochs=2)

Epoch 1/2
10000/10000 [==============================] - 348s 35ms/step - loss: 14.5141
Epoch 2/2
10000/10000 [==============================] - 346s 35ms/step - loss: 14.5224
